In [6]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
import imgaug


In [7]:
# Root directory of the project

ROOT_DIR = r"C:\Users\92321\Threads of Fashion"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



In [8]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"


    # NUMBER OF GPUs to use. When using only a CPU, this needs to be set to 1.
    GPU_COUNT = 1
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 4
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 11  # Background + Hard_hat, Safety_vest

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 2

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    LEARNING_RATE = 0.001

In [9]:

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
      
        self.add_class("object", 1, "Bridal Long Shirt")
        self.add_class("object", 2, "Bridal Dupatta")
        self.add_class("object", 3, "Bridal Choli")
        self.add_class("object", 4, "Bridal Lehenga")
        self.add_class("object", 5,"Wedding Guest Lehenga")
        self.add_class("object", 6,"Wedding Guest Dupatta")
        self.add_class("object", 7,"Wedding Guest Choli")
        self.add_class("object", 8,"Bridal Maxi")
        self.add_class("object", 9,"Wedding Guest Long Shirt")
        self.add_class("object", 10,"Bridal Peplum")
        self.add_class("object", 11,"Wedding Guest Maxi")
        
        
        


     
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        if subset == "train":
            annotations1 = json.load(open(r'C:\Users\92321\Threads of Fashion\dataset\train\train.json'))
        elif subset == "val":
            annotations1 = json.load(open(r'C:\Users\92321\Threads of Fashion\dataset\val\val.json'))

        annotations = list(annotations1.values())
        print(annotations1)
       
          # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:

            polygons = [r['shape_attributes'] for r in a['regions'].values()] 
            objects = [s['region_attributes'] for s in a['regions'].values()]
            print("objects:",objects)
            num_ids=[]
            

            for n in objects:

                try:

                    if n['name']== "Bridal Long Shirt":
                        num_ids.append(1)

                    elif n['name']=="Bridal Dupatta":
                         num_ids.append(2)
                    
                    elif n['name']=="Bridal Choli":
                         num_ids.append(3)

                    elif n['name']=="Bridal Lehenga":
                         num_ids.append(4)
                    
                    elif n['name']=="Wedding Guest Lehenga":
                         num_ids.append(5)
                    
                    elif n['name']=="Wedding Guest Dupatta":
                         num_ids.append(6)

                    elif n['name']=="Wedding Guest Choli":
                         num_ids.append(7)
                    
                    elif n['name']=="Bridal Maxi":
                         num_ids.append(8)

                    elif n['name']=="Wedding Guest Long Shirt":
                         num_ids.append(9)
                    
                    elif n['name']=="Bridal Peplum":
                         num_ids.append(10)
                    
                    elif n['name']=="Wedding Guest Maxi":
                         num_ids.append(11)
                    
                except:
                    
                    pass




                    



            
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
           
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(r"C:\Users\92321\Threads of Fashion\dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(r"C:\Users\92321\Threads of Fashion\dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    
    # print("Training network heads")
    # model.train(dataset_train, dataset_val,
                # learning_rate=config.LEARNING_RATE,
                # epochs=250,
                # layers='heads')
                
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=2,
                layers='heads', #layers='all', 
                augmentation = imgaug.augmenters.Sequential([ 
                imgaug.augmenters.Fliplr(0.5), 
                imgaug.augmenters.Flipud(0.5), 
                imgaug.augmenters.Affine(rotate=(-45, 45)), 
                imgaug.augmenters.Affine(rotate=(-90, 90)), 
                imgaug.augmenters.Affine(scale=(0.5, 1.5)),
                imgaug.augmenters.Crop(px=(0, 10)),
                imgaug.augmenters.Grayscale(alpha=(0.0, 1.0)),
                imgaug.augmenters.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                imgaug.augmenters.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                imgaug.augmenters.Invert(0.05, per_channel=True), # invert color channels
                imgaug.augmenters.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                
                ])
                )
                
            



'''
 this augmentation is applied consecutively to each image. In other words, for each image, the augmentation apply flip LR,
 and then followed by flip UD, then followed by rotation of -45 and 45, then followed by another rotation of -90 and 90,
 and lastly followed by scaling with factor 0.5 and 1.5. '''
	
    
# Another way of using imgaug    
# augmentation = imgaug.Sometimes(5/6,aug.OneOf(
                                            # [
                                            # imgaug.augmenters.Fliplr(1), 
                                            # imgaug.augmenters.Flipud(1), 
                                            # imgaug.augmenters.Affine(rotate=(-45, 45)), 
                                            # imgaug.augmenters.Affine(rotate=(-90, 90)), 
                                            # imgaug.augmenters.Affine(scale=(0.5, 1.5))
                                             # ]
                                        # ) 
                                   # )
                                    

    
				


'\n this augmentation is applied consecutively to each image. In other words, for each image, the augmentation apply flip LR,\n and then followed by flip UD, then followed by rotation of -45 and 45, then followed by another rotation of -90 and 90,\n and lastly followed by scaling with factor 0.5 and 1.5. '

In [10]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)			

{'120036931_372909770518626_6948827444533006010_n.jpg250199': {'fileref': '', 'size': 250199, 'filename': '120036931_372909770518626_6948827444533006010_n.jpg', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [330, 330, 335, 381, 455, 517, 588, 571, 529, 514, 514, 522, 556, 593, 647, 593, 519, 494, 443, 379, 337, 106, 86, 69, 2, 2, 49, 106, 148, 221, 271, 362, 408, 413, 379, 308, 256, 204, 150, 89, 62, 44, 111, 177, 229, 276, 330], 'all_points_y': [273, 273, 271, 298, 271, 423, 534, 554, 541, 586, 649, 753, 859, 1046, 1304, 1316, 1316, 1311, 1326, 1289, 1314, 1242, 1171, 1201, 1191, 1031, 967, 932, 888, 755, 677, 608, 588, 566, 497, 394, 467, 534, 586, 649, 652, 642, 551, 463, 384, 295, 273]}, 'region_attributes': {'name': 'Bridal Long Shirt'}}, '1': {'shape_attributes': {'name': 'polygon', 'all_points_x': [465, 497, 539, 546, 541, 588, 588, 662, 755, 873, 947, 1075, 1075, 979, 923, 871, 832, 792, 709, 645, 578, 51

c:\Users\92321\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
c:\Users\92321\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub_1:0", shape=(None,), 

2/2 [==============================] - ETA: 0s - batch: 0.5000 - size: 4.0000 - loss: 6.3340 - rpn_class_loss: 0.0342 - rpn_bbox_loss: 1.1226 - mrcnn_class_loss: 2.2658 - mrcnn_bbox_loss: 1.3805 - mrcnn_mask_loss: 1.5310      

c:\Users\92321\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


2/2 [==============================] - 542s 420s/step - batch: 0.5000 - size: 4.0000 - loss: 6.3340 - rpn_class_loss: 0.0342 - rpn_bbox_loss: 1.1226 - mrcnn_class_loss: 2.2658 - mrcnn_bbox_loss: 1.3805 - mrcnn_mask_loss: 1.5310 - val_loss: 4.0436 - val_rpn_class_loss: 0.0177 - val_rpn_bbox_loss: 0.6079 - val_mrcnn_class_loss: 1.0653 - val_mrcnn_bbox_loss: 1.2832 - val_mrcnn_mask_loss: 1.0696
Epoch 2/2
2/2 [==============================] - 507s 403s/step - batch: 0.5000 - size: 4.0000 - loss: 3.8388 - rpn_class_loss: 0.0392 - rpn_bbox_loss: 0.9867 - mrcnn_class_loss: 0.6317 - mrcnn_bbox_loss: 1.1244 - mrcnn_mask_loss: 1.0568 - val_loss: 3.3941 - val_rpn_class_loss: 0.0172 - val_rpn_bbox_loss: 0.6263 - val_mrcnn_class_loss: 0.7069 - val_mrcnn_bbox_loss: 1.1380 - val_mrcnn_mask_loss: 0.9057
